<a href="https://colab.research.google.com/github/tsakailab/prml/blob/master/ipynb/ex_MNIST_LDAembedding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LDAによる低次元化（dimensionality reduction by the linear discriminant analysis）

MNIST画像集合を線形判別分析（LDA）
で識別的な低次元空間に埋め込み，観察します．

参考資料：
- [Linear discriminant analysis@Wikipedia](https://en.wikipedia.org/wiki/Linear_discriminant_analysis)
- [Linear and Quadratic Discriminant Analysis@scikit-learn](https://scikit-learn.org/stable/modules/lda_qda.html)
- [Linear Discriminant Analysis – Bit by Bit](https://sebastianraschka.com/Articles/2014_python_lda.html)

----

氏名：

学生番号：

----
## 基本課題（必須）

    1. 手書き数字画像MNISTの 1，4，7，9 の4クラスを線形判別分析で低次元化したとき，
       第1成分（comp. 1）はこれらのクラスをどのように分類するために役立つ特徴量ですか．
       同様に，第2成分（comp. 2）と第3成分（comp. 3）は，それぞれどのクラスについて識別的な特徴量であると言えますか．理由と共に回答してください．

（ここに回答を書いてください）



    2. Fashion-MNISTについて，線形判別分析でどのクラスが他のクラスと識別し易い・し難いですか．低次元空間における分布と混同行列に基づき調べてください．
       また，識別し易い・し難い原因についても考察してください．

（ここに回答を書いてください）



    3. 上記の参考資料等をもとに，線形判別分析による低次元化の原理について調査し，理解できた範囲で解説してください．

（ここに回答を書いてください）



----
発展課題（任意）がこのノートブックの後半にあります．

### データ集合を取得します．

In [ ]:
from torchvision import datasets

# [MNIST](http://yann.lecun.com/exdb/mnist/)
tvds = datasets.MNIST('./data', download=True, train=True)

# [Fashion-MNIST](https://github.com/zalandoresearch/fashion-mnist)
#tvds = datasets.FashionMNIST('./data', download=True, train=True)

# [Kuzushiji-MNIST](https://github.com/rois-codh/kmnist)
#tvds = datasets.KMNIST('./data', download=True, train=True)

Ximages_all, y_all = tvds.data.numpy(), tvds.targets.numpy()

# [EMNIST](https://pytorch.org/vision/stable/generated/torchvision.datasets.EMNIST.html)
#tvds = datasets.EMNIST('./data', download=True, train=True, split='letters')
#Ximages_all, y_all = tvds.data.transpose_(-1,-2).numpy(), tvds.targets.numpy() - 1

In [ ]:
#@title ☆画像数とサイズを確認します（画像を加工したい場合はこのセルを編集して利用してください）．

# simulate noisy images
#Ximages_all = Ximages_all + np.random.rand(*Ximages_all.shape) * 200
#Ximages_all[Ximages_all > 255] = 255

import numpy as np
from skimage.filters import gaussian
from skimage.exposure import equalize_hist as eh
from skimage.transform import resize

'''
* blurring (https://scikit-image.org/docs/stable/api/skimage.filters.html#skimage.filters.gaussian)
* histogram equalization (https://scikit-image.org/docs/stable/auto_examples/color_exposure/plot_equalize.html)
* resize images (https://scikit-image.org/docs/stable/auto_examples/transform/plot_rescale.html)
'''
#Ximages_all = gaussian(np.float32(Ximages_all.transpose((1,2,0))), sigma=1.0, multichannel=True).transpose(2,0,1)
#Ximages_all = eh(Ximages_all.transpose((1,2,0))).transpose(2,0,1) * 255
#height, width = 8, 8; Ximages_all = resize(Ximages_all.transpose((1,2,0)), (height, width)).transpose(2,0,1)

Ximages_all = np.uint8(Ximages_all)
print("(#images, height, width)", Ximages_all.shape)
print(Ximages_all.dtype, ", max. pixel value = ", Ximages_all.max())

MNIST class labels

| [MNIST](http://yann.lecun.com/exdb/mnist/) | 0 | 1 | 2 | 3 | 4 | 5 | 6 | 7 | 8 | 9 |
| - | :-: | :-: | :-: | :-: | :-: | :-: | :-: | :-: | :-: | :-: |
| [Fashion-MNIST](https://github.com/zalandoresearch/fashion-mnist) | **T-shirt/top** | Trouser | **Pullover** | Dress | **Coat** | Sandal |  **Shirt** | Sneaker | Bag | Ankle boot |
| [Kuzushiji-MNIST](https://github.com/rois-codh/kmnist) | お | き | す | つ | な | は |  ま | や | れ | を |

In [ ]:
#@title 画像を例示します．
#@title Show images of digits
import numpy as np
from matplotlib import pyplot as plt
print("%d images in total" % len(y_all))

# show the digits
def plotMNIST(imgs, gts, preds=None, num=8, mag=0.6):
    classes = np.unique(gts)
    nc = len(classes)
    idx = np.arange(len(gts))
    fig = plt.figure(figsize=(num*mag, nc*mag))  # figure size in inches
    fig.subplots_adjust(left=0, right=1, bottom=0, top=1, hspace=0.05, wspace=0.05)
    for i, c in enumerate(classes):
        imgsc = imgs[gts==c]
        idc = idx[gts==c]
        for j in range(min(num,len(imgsc))):
            ax = fig.add_subplot(nc, num, i*num+j+1, xticks=[], yticks=[])
            ax.imshow(imgsc[j], cmap=plt.cm.gray, interpolation='nearest')

            # label the image with the target value
            ax.text(0, imgs.shape[1]*0.2, str(c), color='white')
            if preds is not None:
                if preds[idc[j]] == c:
                    ax.text(imgs.shape[2]*0.8, imgs.shape[1]*0.2, str(preds[idc[j]]), color='#a0ffa0')
                else:
                    ax.text(imgs.shape[2]*0.8, imgs.shape[1]*0.2, str(preds[idc[j]]), color='red')

p = np.random.permutation(len(y_all))
plotMNIST(Ximages_all[p], y_all[p], num=16)

### 4クラスの識別問題にしたい場合（さもなくば実行しなくてよいです）
- このセルの `c0` ～ `c3` でクラスを指定して実行してください．

In [ ]:
c0 = 1 # choose from 0 to 9
c1 = 4 # choose from 0 to 9
c2 = 7 # choose from 0 to 9
c3 = 9 # choose from 0 to 9

Ximages = Ximages_all[np.logical_or.reduce((y_all == c0, y_all == c1, y_all == c2, y_all == c3))]
y = y_all[np.logical_or.reduce((y_all == c0, y_all == c1, y_all == c2, y_all == c3))]
classes = np.unique(y)

p = np.random.permutation(len(y))
plotMNIST(Ximages[p], y[p], num=20)

### すべてのクラスを使いたい場合

In [ ]:
Ximages = Ximages_all
y = y_all
classes = np.unique(y)

### ☆訓練データと検証データに分けます．
- 画像を `Ximages_train` と `Ximages_val` に分けます．それぞれ `n_train` 枚と `n_val` 枚の画像です．
- `Ximages_train` と `Ximages_val` を，それぞれ shape が `(n_train, 画素数)`，`(n_val，画素数)` の NumPy 配列にしたものが `X_train`，`X_val` です．

In [ ]:
from sklearn.model_selection import train_test_split

# split the data into training and validation sets
Ximages_train, Ximages_val, y_train, y_val = train_test_split(Ximages, y, train_size=0.8)

n_train = len(Ximages_train)
n_val = len(Ximages_val)

# reshape to 28*28=784-dimensional feature vectors
X_train = np.reshape(Ximages_train, (Ximages_train.shape[0], -1)) / 255   # (n_train, 784)
X_val = np.reshape(Ximages_val, (Ximages_val.shape[0], -1)) / 255         # (n_val, 784)
print("X_train.shape = ", X_train.shape)
print("X_val.shape = ", X_val.shape)

### ★線形判別分析を実行します（**2次元化の場合**）．
- [sklearn.discriminant_analysis.LinearDiscriminantAnalysis](https://scikit-learn.org/stable/modules/generated/sklearn.discriminant_analysis.LinearDiscriminantAnalysis.html)を使用します．引数の仕様とデフォルトの値を確認してください．

In [ ]:
#@title 2次元データ X を y で色分けして正方形内にプロットする関数 plot_embedding2d(X, y) を定義します．
# https://scikit-learn.org/stable/auto_examples/manifold/plot_lle_digits.html#helper-function-to-plot-embedding

import numpy as np
from matplotlib.colors import TwoSlopeNorm as tsn
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler

def plot_embedding2d(X, y, title="", xlabel="", ylabel="", axis="on", xlim=[-3.1, 3.1], ylim=[-3.1, 3.1], Scaler=None):
    _, ax = plt.subplots()

    if Scaler is None:
        Scaler = RobustScaler() # MinMaxScaler(), StandardScaler()
        X2d = Scaler.fit_transform(X[:,:2])
    else:
        X2d = Scaler.transform(X[:,:2])

    classes = np.unique(y)
    for yi in classes:
        ax.scatter(
            *X2d[y == yi].T,
            marker=f"${yi}$",
            s=60,
            color=plt.cm.tab10(yi), #Paired(yi),
            alpha=0.425,
            zorder=2,
        )

    ax.set_xlim(xlim)
    ax.set_ylim(ylim)
    ax.set_title(title)
    ax.set_xlabel(xlabel)
    ax.set_ylabel(ylabel)
    ax.axis(axis)
    ax.set_aspect(1)

    return Scaler

#####imshows(model.scalings_[:,:2].T.reshape(-1, Ximages.shape[1], Ximages.shape[2]), robust=True)
def imshows(imgs, mag=1.0, cmap=None, robust=True): # imgs (num, height, width)
    if cmap is None:
        cmap = plt.cm.seismic

    if robust:
        imgsc = RobustScaler().fit_transform(imgs.reshape(imgs.shape[0],-1)).reshape(-1,imgs.shape[1], imgs.shape[2])
    else:
        imgsc = imgs

    nc = imgsc.shape[0]
    fig = plt.figure(figsize=(nc*mag, mag))
    fig.subplots_adjust(left=0, right=1, bottom=0, top=1, hspace=0.05, wspace=0.05)

    vmin, vmax = np.minimum(imgsc.min(),-1e-6), np.maximum(imgsc.max(),1e-6)
    print(vmin, vmax)
    norm = tsn(vmin=vmin, vcenter=0.5, vmax=vmax)
    for i in range(nc):
        ax = fig.add_subplot(1, nc, i + 1, xticks=[], yticks=[])
        ax.imshow(imgsc[i], cmap=cmap, norm=norm, vmin=vmin,vmax=vmax)
        ax.set_title(str(i))

In [ ]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
model = LinearDiscriminantAnalysis(n_components=2)

Xe_train = model.fit_transform(X_train, y_train)
Xe_val = model.transform(X_val)

In [ ]:
# 全部表示すると多いので，ランダムに n 個だけ表示する．
nt = min(100, n_train)
pt = np.random.choice(X_train.shape[0], nt, replace=False)

nv = min(100, n_val)
pv = np.random.choice(X_val.shape[0], nv, replace=False)

sc = plot_embedding2d(Xe_train[pt], y_train[pt], title="train", xlabel="comp. 1", ylabel="comp. 2")
plot_embedding2d(Xe_val[pv], y_val[pv], Scaler=sc, title="val", xlabel="comp. 1", ylabel="comp. 2")

import matplotlib.pyplot as plt
plt.show()

### ★線形判別分析を実行します（**3次元化の場合**）．
- [sklearn.discriminant_analysis.LinearDiscriminantAnalysis](https://scikit-learn.org/stable/modules/generated/sklearn.discriminant_analysis.LinearDiscriminantAnalysis.html)を使用します．引数の仕様とデフォルトの値を確認してください．

In [ ]:
#@title 3次元データ X を y で色分けして立方体内にプロットする関数 plot_embedding3d(X, y) を定義します．
%matplotlib inline
import numpy as np
from matplotlib import offsetbox
from sklearn.preprocessing import MinMaxScaler

import plotly.graph_objs  as go
import plotly.graph_objs.layout  as gol
import plotly.io as pio
pio.renderers.default = 'colab'

def plot_embedding3d(X, y, xlim=[-3.1, 3.1], ylim=[-3.1, 3.1], zlim=[-3.1,3.1], Scaler=None):
    if Scaler is None:
        Scaler = RobustScaler() # MinMaxScaler(), StandardScaler()
        X3d = Scaler.fit_transform(X[:,:3])
    else:
        X3d = Scaler.transform(X[:,:3])

    # https://plotly.com/python-api-reference/generated/plotly.graph_objects.Scatter3d.html#plotly.graph_objects.Scatter3d
    trace = go.Scatter3d(x=X3d[:,0], y=X3d[:,1], z=X3d[:,2], mode='text',
                         text=y, textfont=dict(color=['rgba({},{},{},{})'.format(c[0],c[1],c[2],0.8) for c in plt.cm.tab10(y)]), textposition='top center'
    )

    layout = go.Layout(margin=dict(l=0,r=0,b=0,t=0), scene=gol.Scene(aspectmode='cube', xaxis=gol.scene.XAxis(title="comp. 1"), yaxis=gol.scene.YAxis(title="comp. 2"), zaxis=gol.scene.ZAxis(title="comp. 3")))
    fig = go.Figure(data=[trace], layout=layout)
    camera = dict(up=dict(x=0, y=0, z=3), center=dict(x=0, y=0, z=0), eye=dict(x=1.5, y=1.5, z=0.8))
    scene = dict(xaxis=dict(range=xlim), yaxis=dict(range=ylim), zaxis=dict(range=zlim))
    fig.update_layout(scene_camera=camera, scene=scene)
    fig.show()
    return Scaler

In [ ]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
model = LinearDiscriminantAnalysis(n_components=3)

Xe_train = model.fit_transform(X_train, y_train)
Xe_val = model.transform(X_val)

# 全部表示すると多いので，ランダムに n 個だけ表示する．
nt = min(300, n_train)
pt = np.random.choice(X_train.shape[0], nt, replace=False)
sc = plot_embedding3d(Xe_train[pt], y_train[pt])

nv = min(300, n_val)
pv = np.random.choice(X_val.shape[0], nv, replace=False)
plot_embedding3d(Xe_val[pv], y_val[pv], Scaler=sc)

In [ ]:
#@title 識別の結果を例示します（上：訓練データ，下：検証データ）．
from sklearn.metrics import accuracy_score

y_pred = model.predict(X_train)
print("Accuracy on training data: ", accuracy_score(y_train, y_pred))
p = np.random.permutation(n_train)
plotMNIST(Ximages_train[p], y_train[p], y_pred[p], num=16)
plt.show()

y_pred = model.predict(X_val)
print("\nAccuracy on validation data: ", accuracy_score(y_val, y_pred))
p = np.random.permutation(n_val)
plotMNIST(Ximages_val[p], y_val[p], y_pred[p], num=16)

MNIST class labels

| [MNIST](http://yann.lecun.com/exdb/mnist/) | 0 | 1 | 2 | 3 | 4 | 5 | 6 | 7 | 8 | 9 |
| - | :-: | :-: | :-: | :-: | :-: | :-: | :-: | :-: | :-: | :-: |
| [Fashion-MNIST](https://github.com/zalandoresearch/fashion-mnist) | T-shirt/top | Trouser | Pullover | Dress | Coat | Sandal |  Shirt | Sneaker | Bag | Ankle boot |
| [Kuzushiji-MNIST](https://github.com/rois-codh/kmnist) | お | き | す | つ | な | は |  ま | や | れ | を |

In [ ]:
#@title 混同行列（行：正解，列：予測）
from sklearn import metrics
# https://scikit-learn.org/stable/modules/generated/sklearn.metrics.confusion_matrix.html

cm = metrics.confusion_matrix(y_val, y_pred)
print(cm)

print(cm.sum(axis=0))

In [ ]:
#@title おまけ
import seaborn as sns

# Make predictions on test data
cm = metrics.confusion_matrix(y_val, y_pred)
cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

plt.figure(figsize=(9,9))
sns.heatmap(cm_normalized, annot=True, fmt=".3f", linewidths=.5, square = True, cmap = 'Blues_r');
plt.ylabel('Actual label');
plt.xlabel('Predicted label');

--------
## 発展課題（任意）

学習済みの畳み込みニューラルネット（DCNN）で画像から抽出した特徴（DCNN特徴）を用いて線形判別分析してみましょう．
画素値のまま線形判別分析に用いるよりも，識別し易くなるクラスがあるかもしれません．

    1. Fashion-MNISTの 0:T-shirt/top, 2:Pullover, 4:Coat, 6:Shirt の4クラスについて，画素値を用いた場合と，DCNN特徴を用いた場合を比較してください．
       識別的な低次元化がし易いのはどちらでしょうか．また，識別の結果はどうですか．

（ここに回答を書いてください）



    2. MedMNISTについても，同様に比較・考察しましょう．

（ここに回答を書いてください）[参考文献](https://arxiv.org/abs/2110.14795)





In [ ]:
#@title 学習済みの畳み込みニューラルネットワークで特徴抽出する関数 feature_extractor を定義します．<p><ul><li>「☆訓練データと検証データに分けます．」のセルを実行後に，下のセルで `X_train` と `X_val` を `feature_extractor` で作成してください．</li><li>`X_train` と `X_val` を作成後，「★線形判別分析を実行します」に戻って，線形判別分析による低次元化を実行してください．</li></ul>
import torch
from torchvision import models

#model_pretrained = models.alexnet(weights='DEFAULT', progress=False)
#model_pretrained = models.vgg16(weights='DEFAULT')
#model_pretrained = models.vgg16_bn(weights='DEFAULT')

#model_pretrained = models.resnet50(weights='DEFAULT')
#model_pretrained = models.googlenet(weights='DEFAULT')
#model_pretrained = models.mobilenet_v3_small(weights='DEFAULT')
model_pretrained = models.efficientnet_b0(weights='DEFAULT')

#feature_extractor = model_pretrained.features

class DCNN(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.features = model_pretrained.features
        self.gap = torch.nn.AdaptiveAvgPool2d(output_size=(1,1))

    def forward(self, x):
        x = self.features(x)
        x = self.gap(x).squeeze(-1).squeeze(-1) # [B, C, H=1, W=1] -> [B, C]
        return x

DCNNfeatures = DCNN()

device = 'cuda' if torch.cuda.is_available() else 'cpu'
DCNNfeatures.eval().to(device)

import torchvision.transforms as transforms

def feature_extractor(Ximages, batch_size=256): # Ximages(n_samples, height, width)
    with torch.no_grad():
        imgs = torch.tensor(Ximages).to(device)
        if Ximages.ndim == 3:
            imgs.unsqueeze_(1)
            imgs = imgs / 255.0
            n, _, h, w = imgs.shape
            imgs = imgs.view(n, 1, h, w).expand(-1,3,-1,-1)
        else:
            imgs = imgs.permute(0,3,1,2)
            imgs = transforms.Normalize(mean = [0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225])(imgs.float())

        dataloader = torch.utils.data.DataLoader(imgs, batch_size=batch_size)        
        f = []
        for bimgs in dataloader:
            bimgs = transforms.Resize(112)(bimgs)
            x = DCNNfeatures(bimgs.to(device))
            f.append(x.cpu().numpy())

    return np.concatenate(f)

In [ ]:
# 特徴抽出
# 注意：数分かかります！ ランタイムのタイプをGPUへ変更することを推奨します．CPUでは，colabのメモリ不足でカーネルがクラッシュすることがあります．
X_train = feature_extractor(Ximages_train)
X_val = feature_extractor(Ximages_val)

In [ ]:
#@title [MedMNIST](https://github.com/MedMNIST/MedMNIST)を使う場合<p>以下を実行後，このJupyter Notebook前半の「☆画像数とサイズを確認します（画像を加工したい場合はこのセルを編集して利用してください）．」以降を実行できます．
!pip install -q medmnist
import medmnist
print(f"MedMNIST v{medmnist.__version__} @ {medmnist.HOMEPAGE}")

In [ ]:
# 'pneumoniamnist', 'chestmnist', 'octmnist', 'breastmnist', 'tissuemnist', 'organamnist', 'organcmnist', 'organsmnist' 
data_flag = 'retinamnist' # 'octmnist' 
DataClass = getattr(medmnist, medmnist.INFO[data_flag]['python_class'])

tvds = DataClass(split='train', download=True)
#tvds = DataClass(split='test', download=True)

print(tvds)
Ximages_all, y_all = tvds.imgs, tvds.labels.ravel()

お疲れさまでした．